In [ ]:
import tensorflow as tf
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from keras import *
from keras.src.layers import Dense, LeakyReLU, Dropout
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
import zipfile

### For data from local disk:

In [ ]:
# If we want to use the models later, need to all have the same version of tensorflow and keras
print(tf.__version__)
print(keras.__version__)

In [ ]:
data_dir = '' # Path to the folder that has good and bad images

image_size = (512,512)

train_data, val_data = keras.utils.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    validation_split=0.2,
    subset="both",
    seed=1354,
    color_mode="rgb",
    image_size=image_size,
    shuffle=True,
)

In [ ]:
# Printing the images
plt.figure(figsize=(3, 3))
for images, labels in train_data.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(images[i]).astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

In [ ]:
# Need to normalise the datat
def normalize_data(data, label):
    data = tf.cast(data, tf.float32) / 255.0
    return data, label

train_data = train_data.map(normalize_data)
val_data = val_data.map(normalize_data)

In [ ]:
# Feel free to change the model up
model = keras.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(512, 512, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'), 
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=['accuracy']) # Could change this to recall

print(model.summary())

In [ ]:
# BEFORE RUNNING THIS:
    # CHANGE THE NAME OF THE FILE BEFORE TRAINING, and keep track of which model is what ()


epochs = 20

callbacks = [
    keras.callbacks.ModelCheckpoint("Surface_Roughness_Cropped3_Dropout0.2_L-Data_512_E{epoch}.keras"), # change the name of the file
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',     # Monitor validation accuracy
        patience=3,                 # Stop after 3 epochs of no improvement
        restore_best_weights=True   # Restore the best weights
    )
]

history = model.fit(
    train_data,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_data,
)

In [ ]:
# Change current description to something that describes the model, so we can tell it apart
current_description = ''
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('CNN Model Loss ' + current_description)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
# plt.ylim(0, 0.6)
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('CNN Model Accuracy ' + current_description)
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.ylim(0, 1)
plt.show()